# Lab 6: 고객 대상 프론트엔드 애플리케이션 구축

## 개요

이전 랩들에서 메모리, 공유 도구, 프로덕션급 배포, 관측성을 갖춘 포괄적인 이커머스 고객 지원 에이전트를 구축했습니다. 이제 고객이 실제로 우리 에이전트와 상호작용할 수 있는 사용자 친화적인 **Streamlit 기반 웹 애플리케이션**을 실행해보겠습니다.

**워크숍 여정:**
- **Lab 1 (완료)**: 에이전트 프로토타입 생성 - 기능적인 이커머스 고객 지원 에이전트 구축
- **Lab 2 (완료)**: 메모리로 강화 - 대화 컨텍스트 및 개인화 추가
- **Lab 3 (완료)**: Gateway & Identity로 확장 - 에이전트 간 도구 안전하게 공유
- **Lab 4 (완료)**: 프로덕션 배포 - AgentCore Runtime
- **Lab 5 (완료)**: AgentCore 관측성 - CloudWatch GenAI 관측성 탐색
- **Lab 6 (현재)**: 사용자 인터페이스 구축 - 고객 대상 애플리케이션 실행

### Lab 6를 위한 아키텍처

우리의 프론트엔드 애플리케이션은 Lab 4에서 배포한 AgentCore Runtime 엔드포인트에 연결되어 완전한 종단 간 이커머스 고객 지원 솔루션을 제공합니다:

<div style="text-align:left">
    <img src="images/architecture_lab6_streamlit.png" width="100%"/>
</div>

*K-Style 이커머스 플랫폼의 완전한 고객 지원 솔루션: 고객이 웹 인터페이스를 통해 상품 문의, 반품 처리, 추천 서비스를 받을 수 있습니다.*

### 제공되는 기능

- **보안 인증** - Amazon Cognito를 통한 자동 사용자 인증
- **실시간 채팅 인터페이스** - Streamlit 기반 대화형 UI
- **빠른 액션 버튼** - 반품, 교환, 스타일링 조언 원클릭 서비스
- **세션 관리** - 메모리와 함께 지속적인 대화
- **응답 시간 측정** - 투명성을 위한 성능 메트릭
- **한국어 UI/UX** - K-Style 브랜드에 맞춤화된 디자인

### 랩 목표

이 랩 완료 후 다음을 경험하게 됩니다:

- 완전한 종단 간 이커머스 고객 지원 솔루션 체험
- 실제 고객 관점에서의 AI 에이전트 상호작용
- 프로덕션 준비된 웹 애플리케이션 인터페이스 확인
- 모든 Lab 1-5의 기능이 통합된 최종 결과물 테스트

## 전제 조건

- **Lab 1-5 완료**
- **AgentCore Runtime이 배포되어 실행 중** (Lab 4에서 배포)
- **환경 설정 완료** (가상환경 및 종속성 설치)

### Step 1: 환경 상태 확인

Streamlit 애플리케이션 실행 전 필요한 환경이 준비되어 있는지 확인하겠습니다.

In [ ]:
# 환경 상태 확인
import os
import subprocess
import sys

print("🔍 Lab 6 실행 환경 확인")
print("=" * 40)

# 프로젝트 루트 디렉토리 확인
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
print(f"✅ 프로젝트 루트: {project_root}")

# Streamlit 앱 파일 확인
app_path = os.path.join(project_root, "src/ui/streamlit_app.py")
if os.path.exists(app_path):
    print(f"✅ Streamlit 앱: {app_path}")
else:
    print(f"❌ Streamlit 앱을 찾을 수 없습니다: {app_path}")

# 실행 스크립트 확인
script_path = os.path.join(project_root, "setup/run_streamlit.sh")
if os.path.exists(script_path):
    print(f"✅ 실행 스크립트: {script_path}")
else:
    print(f"⚠️ 실행 스크립트를 찾을 수 없습니다: {script_path}")

print("\n🎯 환경 확인 완료!")

### Step 2: Lab 4 AgentCore Runtime 상태 확인

프론트엔드가 연결할 AgentCore Runtime이 실행 중인지 확인합니다.

In [ ]:
# Lab 4 Runtime 상태 확인
import sys
sys.path.insert(0, '..')
from src.helpers.utils import get_ssm_parameter
import boto3
import json

print("🔍 Lab 4 AgentCore Runtime 상태 확인")
print("=" * 40)

boto_session = boto3.session.Session()
region = boto_session.region_name

# AgentCore Runtime ARN 확인
try:
    runtime_arn = get_ssm_parameter("/app/ecommerce/agentcore/runtime_arn")
    print(f"✅ Runtime ARN 발견: {runtime_arn}")
    
    # boto3로 Runtime 상태 확인
    agentcore_client = boto3.client('bedrock-agentcore', region_name=region)
    
    response = agentcore_client.get_runtime_endpoint(
        runtimeArn=runtime_arn
    )
    status = response.get('status', 'UNKNOWN')
    print(f"✅ Runtime 상태: {status}")
    
    if status == "READY":
        print("🎯 AgentCore Runtime이 준비되었습니다!")
    else:
        print(f"⚠️ Runtime 상태가 READY가 아닙니다: {status}")
        print("   Lab 4에서 Runtime이 완전히 배포될 때까지 기다려주세요.")
        
except Exception as e:
    print(f"❌ Runtime 확인 실패: {e}")
    print("   Lab 4를 먼저 완료하고 Runtime을 배포해주세요.")

# Cognito 설정 확인
try:
    secrets_client = boto3.client('secretsmanager')
    secret_response = secrets_client.get_secret_value(SecretId="ecommerce_customer_support_agent")
    cognito_config = json.loads(secret_response['SecretString'])
    print(f"✅ Cognito 설정 발견: User Pool {cognito_config['pool_id']}")
except Exception as e:
    print(f"❌ Cognito 설정을 찾을 수 없습니다: {e}")

### Step 3: K-Style 고객 지원 애플리케이션 실행 🚀

이제 준비된 Streamlit 애플리케이션을 실행해보겠습니다. 이미 모든 기능이 구현되어 있는 완성된 애플리케이션입니다.

**애플리케이션 특징:**
- Lab 4에서 배포한 AgentCore Runtime에 자동 연결
- Amazon Cognito 기반 자동 인증
- 한국어 UI/UX (K-Style 브랜드)
- 실시간 채팅 인터페이스
- 반품, 교환, 스타일링 조언 빠른 액션
- 응답 시간 측정 및 성능 메트릭
- 세션 메모리 및 대화 맥락 유지

#### 🚀 K-Style 고객 지원 애플리케이션 실행 가이드

**실행 방법:**

1️⃣ **새 터미널을 열고 프로젝트 디렉토리로 이동:**
```bash
cd /home/ubuntu/Self-Study-Generative-AI/lab/18_ec-customer-support-agent-bedrock_agent_core
```

2️⃣ **준비된 실행 스크립트 실행:**
```bash
./run_streamlit_customer_suppot_agent.sh
```

3️⃣ **브라우저에서 제공된 URL로 접속:**
```
http://localhost:8501
```

---

#### 🧪 테스트 시나리오:

- 🛍️ **상품 문의**: "최신 겨울 코트 컬렉션 어떤 게 있나요?"
- 📦 **반품 신청**: "지난주에 주문한 원피스 사이즈가 안 맞아서 반품하고 싶어요"
- 🔄 **교환 문의**: "주문한 니트 색상을 다른 색으로 교환 가능한가요?"
- 💄 **스타일링**: "30대 직장 여성에게 어울리는 겨울 코디 추천해주세요"
- 📞 **주문 조회**: "주문번호 ORD-20240810-001 배송 상태를 확인해주세요"
- 🎯 **개인화**: "평소 선호하는 스타일을 기억해서 추천해주세요"

#### 🔍 확인할 주요 기능:

- ✅ 자동 Cognito 인증 (별도 로그인 불필요)
- ✅ K-Style 브랜드 UI/UX (한국어 인터페이스)
- ✅ 빠른 액션 버튼 (반품, 교환, 스타일링)
- ✅ 실시간 채팅 및 응답 시간 표시
- ✅ 대화 메모리 유지 (이전 대화 기억)
- ✅ 도구 통합 (반품 처리, 상품 추천 등)
- ✅ 세션 연속성 (페이지 새로고침 후에도 기억)
- ✅ 사이드바 정보 (운영시간, 연락처 등)

#### ⚠️ 참고사항:

- **애플리케이션 중지**: 터미널에서 `Ctrl+C`
- **환경 문제 발생 시**: `setup/create_kstyle_env.sh` 재실행
- **Runtime 연결 실패 시**: Lab 4 Runtime 상태 확인
- **Cognito 토큰 만료 시**: 애플리케이션 재시작

## 🎉 Lab 6 완료!

축하합니다! **완전한 종단 간 이커머스 고객 지원 솔루션**을 성공적으로 체험했습니다!

### 🏆 전체 워크숍 완주 달성!

Lab 1부터 Lab 6까지의 여정을 통해 다음과 같은 완전한 AI 에이전트 시스템을 구축했습니다:

**🔄 완성된 시스템 아키텍처:**

1. **Lab 1**: 🤖 **AI 에이전트 프로토타입** - 반품, 교환, 추천 도구를 갖춘 기본 에이전트
2. **Lab 2**: 🧠 **메모리 통합** - 고객별 대화 기억 및 개인화 서비스
3. **Lab 3**: 🌐 **Gateway & Identity** - 안전한 도구 공유 및 엔터프라이즈 확장
4. **Lab 4**: 🚀 **프로덕션 배포** - AgentCore Runtime으로 자동 확장
5. **Lab 5**: 📊 **관측성** - CloudWatch GenAI 관측성으로 모니터링
6. **Lab 6**: 💻 **고객 인터페이스** - 실제 고객이 사용하는 웹 애플리케이션

### 🎯 달성한 비즈니스 가치:

✅ **24/7 자동화된 고객 지원**: 인간 상담사 없이도 전문적인 서비스  
✅ **개인화된 고객 경험**: 고객별 선호도 학습 및 맞춤 서비스  
✅ **운영 비용 절감**: AI로 반복적인 문의 자동 처리  
✅ **확장 가능한 아키텍처**: 수천 명 동시 사용자 지원  
✅ **엔터프라이즈급 보안**: Cognito 인증 및 IAM 기반 접근 제어  
✅ **완전한 관측성**: CloudWatch 통합 모니터링 및 성능 추적  

### 🚀 프로덕션 준비 완료:

이제 이 시스템은 **실제 이커머스 비즈니스에 바로 적용**할 수 있는 수준입니다:

- **고객 대상 배포**: 실제 고객에게 서비스 제공 가능
- **비즈니스 확장**: 다양한 상품 카테고리 및 서비스로 확장
- **글로벌 진출**: 다국어 지원 및 현지화 적용
- **데이터 기반 개선**: 수집된 대화 데이터로 지속적 개선

### 💡 다음 발전 방향:

- **음성 인터페이스**: 음성으로 상담받는 기능 추가
- **AR/VR 통합**: 가상 피팅 및 메이크업 체험
- **소셜 커머스**: SNS 플랫폼 통합 및 라이브 쇼핑
- **예측 분석**: 고객 행동 예측 및 선제적 서비스

---

**🎊 축하합니다! Amazon Bedrock AgentCore를 활용한 차세대 이커머스 AI 에이전트 시스템 구축을 완료했습니다!**

여러분은 이제 **AI 에이전트 개발의 전체 생명주기**를 경험하고, **프로덕션 준비된 실제 시스템**을 구축할 수 있는 역량을 갖추게 되었습니다. 🛍️✨